In [1]:
import mujoco
import mujoco.rollout as rollout
from mujoco import sysid
import numpy as np
import matplotlib.pyplot as plt
import mediapy as media
from absl import logging
import base64
from IPython.display import IFrame

logging.set_verbosity("INFO")

def display_report(report):
    html_b64 = base64.b64encode(report.build().encode()).decode()
    return IFrame(src=f"data:text/html;base64,{html_b64}", width="100%", height=800)

In [2]:
H1_2_XML = ""
spec = mujoco.MjSpec.from_string(H1_2_XML)
model = spec.compile()
data = mujoco.MjData(model)

duration = 3.0
n_steps = int(duration / model.opt.timestep)
t = np.arange(n_steps) * model.opt.timestep

ctrl = (5.0 * np.sin(2 * np.pi * 1.5 * t)
        + 3.0 * np.sin(2 * np.pi * 3.7 * t)).reshape(-1, 1)

initial_state = sysid.create_initial_state(model, data.qpos, data.qvel, data.act)

state, sensor = rollout.rollout(model, data, initial_state, ctrl[:-1])
state = np.squeeze(state, axis=0)
sensor = np.squeeze(sensor, axis=0)
times = state[:, 0]

ValueError: XML parse error 8:
Error=XML_ERROR_PARSING_TEXT ErrorID=8 (0x8) Line number=1
